<a href="https://colab.research.google.com/github/dcheung11/chess.com-calculator-python/blob/main/chess_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [122]:
!pip install chess.com

from chessdotcom import get_leaderboards, get_player_stats, get_player_game_archives
import pprint
import requests
import numpy as np
printer = pprint.PrettyPrinter()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Get Top Wins



In [123]:
# Get archived games
def get_monthly_games(archive):
    response = requests.get(archive)
    if response.status_code == 200:
        data = response.json()
        return data['games']
        # do something with the data here
    else:
        print(f'Request failed with status code {response.status_code}')
        return

# Get archives games from last 12 months
def get_game_archives(username,timeclass):
    all_games = []
    data = get_player_game_archives(username).json
    for archive in data["archives"][-12:]:
        monthly_games = get_monthly_games(archive)
        all_games.extend(monthly_games)
    bullet_games = [game for game in all_games if game['time_class'] == timeclass]
    return bullet_games

# Get top wins
def get_top_wins(games,username):
    wins =0
    top_wins = {}
    for game in games:
        is_white_winner = game["white"]["username"] == username  and game["white"]["result"] == 'win'
        is_black_winner = game["black"]["username"] == username and game["black"]["result"] == 'win'
        if is_white_winner:
            wins += 1
            top_wins[game['black']['username']] = game['black']['rating']
        elif is_black_winner:
            wins += 1
            top_wins[game['white']['username']] = game['white']['rating']
    sorted_dict = dict(sorted(top_wins.items(), key=lambda x: x[1], reverse=True))
    return sorted_dict

# games = get_game_archives("damedolla5","bullet")
# top_wins = get_top_wins(games,"damedolla5")
# print(top_wins)
# print("jadhay" in top_wins)

In [124]:
def display_path(users, target):
    if target in users:     
        path = users[0] + " beat "
        current_user = users[1]
        if len(users) > 2:
          for user in users[2:]:
              path += current_user + " who beat "
              current_user = user
              if user == target:
                  path += user
          return path + "\nSeperated by " + str(len(users)-1) + " players"
        return path + current_user + str(len(users)-1) + "\nSeperated by  players"
    else:
        return "ERROR: " + target + " not found in path list"

visited = set()
player_found = False

def search_for_player(username, path, max_path, visited_players, target, timeclass, last_rating):
    global player_found

    if timeclass not in ["daily", "rapid", "blitz", "bullet"]:
      return "Invalid time class. Input one of the following: daily, rapid, blits, bullet"
    # End recursive search when the target has been found
    if player_found:
        return
    # End search if max depth reached
    if path > max_path:
        print("Max search depth reached")
        return
    # End search if already visited
    if username in visited:
        return
    visited.add(username)

    # "Location: {0:20} Revision {1}".format(Location, Revision)
    print("Searching top " + timeclass + " wins for " + "{0:20}".format(username)+ str(last_rating) + " with an end goal of " + target)

    # print("Searching top " + timeclass + " wins for " + username + " " + str(last_rating) + " with an end goal of " + target)
    
    games = get_game_archives(username,timeclass)
    top_wins = get_top_wins(games, username)
    # print(top_wins)
    visited_players.append(username)

    if target in top_wins:
        visited_players.append(target)
        print("-" * 150)
        print(display_path(visited_players, target))  # add rating to the output log
        print("-" * 150)
        player_found = True
    else:
        for key in top_wins.keys():

            search_for_player(key, path+1, max_path, visited_players, target, timeclass, top_wins[key])
            




In [125]:

def transitive_chess_search(your_username, target_username, timeclass):
  print("Running program to check if " + your_username + " has technically beaten " + target_username + " in " + timeclass + " chess")
  print("-" * 150)
  search_for_player(your_username, 0, 50, [], target_username, timeclass, "    ")
  print("Done")



**USAGE**

Fill out the fields and run all

In [127]:
your_username = "damedolla5" 
target_username = "MagnusCarlsen"
timeclass = "daily" # Options: bullet, blitz, rapid, daily
transitive_chess_search(your_username, target_username, timeclass)

Running program to check if damedolla5 has technically beaten MagnusCarlsen in daily chess
------------------------------------------------------------------------------------------------------------------------------------------------------
Done
